<a href="https://www.kaggle.com/code/yagizhidircan/web-scraping-and-regression-in-house-price?scriptVersionId=130541475" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from bs4 import BeautifulSoup as bts
import requests
import pandas as pd

In [ ]:
def getAndParseURL(url):
    result = requests.get(url, headers = {"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, "html.parser")
    return soup

In [ ]:
getAndParseURL("https://www.zingat.com/bozcaada-satilik")

In [ ]:
def lastpagenumber(url):
    soup = getAndParseURL(url)
    lpn = soup.find('span',{'class':'zng-pagination-link-text zng-pagination-mobile-visible'}).text.split(' ')[2]
    return lpn

In [ ]:
lastpagenumber("https://www.zingat.com/kadikoy-satilik")

In [ ]:
linklist = []
def createlinklist(url):
    for i in range (0,int(lastpagenumber(url))+1):
        linklist.append(url+ '?page=' +str(i))
    return len(linklist)

In [ ]:
createlinklist("https://www.zingat.com/cesme-satilik")
createlinklist("https://www.zingat.com/dikili-satilik")
createlinklist("https://www.zingat.com/foca-satilik")
createlinklist("https://www.zingat.com/karaburun-satilik")
createlinklist("https://www.zingat.com/urla-satilik")
createlinklist("https://www.zingat.com/kas-satilik")
createlinklist("https://www.zingat.com/mugla-satilik")
createlinklist("https://www.zingat.com/balikesir-satilik")
createlinklist("https://www.zingat.com/mugla-satilik")

In [ ]:
TITLE = []
PRICE = []
LOCATION = []
AREA = []
ROOM = []

for i in linklist[::]:
    soups = getAndParseURL(i)

    for title in soups.find_all("div", {'class':"zlc-title"}):
        TITLE.append(title.text.split("\n")[0])

    for price in soups.find_all("div",{"class":"feature-item feature-price"}):
        PRICE.append(int(price.text.split(" ")[0].replace(".","")))

    for location in soups.find_all("div",{"class":"zlc-location"}):
        LOCATION.append(location.text.split("\n")[1])

    for area in soups.find_all("div",{"class":"zlc-tags"}):
        AREA.append(area.text.split("\n")[1])

    for room in soups.find_all("div",{"class":"zlc-tags"}):
        ROOM.append(room.text.split("\n")[0])

In [ ]:
df = pd.DataFrame(list(zip(TITLE, PRICE, LOCATION, ROOM, AREA)), columns=["TITLE", "PRICE", "LOCATION", "ROOM", "AREA"])

In [ ]:
df.to_csv(r'file location\yazlik.csv', index=False)

In [ ]:
df.head()

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.head(3)

In [ ]:
df['AREA_M2'] = df.AREA.str.extract('(.*) ')
#df.AREA.append(int(price.text.strip("m²")))
df['TOWN'] = df.LOCATION.str.extract('(.*),')
df['DISTRICT'] = df.LOCATION.str.extract(',(.*)')
df['ROOM1'] = df.ROOM.str.extract('(.)')
df['LOUNGE'] = df.ROOM.str[2]

In [ ]:
df.head()

In [ ]:
df.drop(columns={'ROOM'}, inplace = True)
df.drop(columns={'AREA'}, inplace = True)
df.drop(columns={'LOCATION'}, inplace = True)

In [ ]:
df.head()

In [ ]:
df['ROOM'] = df.ROOM1
df.drop(columns={'ROOM1'}, inplace = True)

In [ ]:
df.head()

In [ ]:
df.ROOM.value_counts()

In [ ]:
df.info()

In [ ]:
df['AREA_M2']= df.AREA_M2.astype(int)
df['ROOM'] = df['ROOM'].fillna(0).astype(int)
df['ROOM'] = df.ROOM.astype(int)
df['LOUNGE'].fillna(0, inplace=True)
df['LOUNGE'].replace(' ', 0, inplace=True)
df['LOUNGE'] = df['LOUNGE'].astype(int)

In [ ]:
null_count = df['LOUNGE'].isnull().sum()
print("Boş değer sayısı:", null_count)

In [ ]:
df.info()

In [ ]:
df['PRICE'].describe().astype(int)

In [ ]:
plt.figure(figsize = [20,5])
sns.boxplot( x = df['PRICE'], data = df)

In [ ]:
percentile_95 = np.percentile(df['PRICE'], 99)
print(percentile_95)

In [ ]:
percentile_5 = np.percentile(df['PRICE'], 5)
print(percentile_5)

In [ ]:

df = df[df.PRICE < 100000000]
df.head(3)

In [ ]:
plt.figure(figsize = [20,5])
sns.boxplot( x = df['PRICE'], data = df)

In [ ]:
df['PRICE'].describe().astype(int)

In [ ]:
df = df[df.PRICE > 3500000]

In [ ]:
#df['PRICE_LOG'] = np.log(df['PRICE'])
#df.PRICE_LOG.describe()
#sns.distplot(df['PRICE'],kde=False)
sns.distplot(df['PRICE'],kde=False)

In [ ]:
#plt.figure(figsize = [20,5])
#sns.boxplot( x = df['PRICE_LOG'], data = df)

In [ ]:
df.PRICE.hist()

In [ ]:
df1.head()

In [ ]:
df1.drop_duplicates(inplace=True)

In [ ]:
df1.info

In [ ]:
df1.columns

In [ ]:
df.head(3)

In [ ]:
X = df.drop(columns=["TITLE", "PRICE"])
y = df.loc[:,"PRICE"]

In [ ]:
X

In [ ]:
y

In [ ]:
df.info()

In [ ]:
df1 = df

In [ ]:
'''df5['DISTRICT'] = pd.get_dummies(df5['DISTRICT'], prefix='DISTRICT')
df5 = pd.concat([df5, df5['DISTRICT'] ], axis=1)'''

In [ ]:
df.columns

In [ ]:
district1 =pd.get_dummies(df1['DISTRICT'],drop_first=True)
df1=pd.concat([df1,district1],axis=1)
df1.head()

In [ ]:
df1.drop(columns={'DISTRICT','TOWN'}, inplace = True)

In [ ]:
df = df1

In [ ]:
df.head()

In [ ]:
#melissabuseozturk = pd.get_dummies(df4['DISTRICT'],drop_first=False)
#melissabuseozturk
#df2=pd.concat([df2,DISTRICT],axis=1)

In [ ]:
df.to_csv(r'file location\yazlik_v7.csv', index=False)

In [ ]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

In [ ]:
# Creating OLS model
housing_model = sm.OLS(y_test,X_test, data=df)

results = housing_model.fit()

# Summary of model
results.summary()

In [ ]:
print("X:", X.shape)
print("y:", y.shape)
print("X_test:", X_test.shape)
print("y_test", y_test.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)

In [ ]:
# Creating the 3 models that we will use for the validation phase (model selection)

# Linear Regression
lm = LinearRegression()

# Ridge Regression
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_val_scaled = scaler.transform(X_val.values)
X_test_scaled = scaler.transform(X_test.values)

lm_reg = Ridge(alpha=1)

# Polinomal Regression
poly = PolynomialFeatures(degree=4) 

X_train_poly = poly.fit_transform(X_train.values)
X_val_poly = poly.transform(X_val.values)
X_test_poly = poly.transform(X_test.values)

lm_poly = LinearRegression()

In [ ]:
lm.fit(X_train, y_train)
print(f'Linear Regression val R^2: {lm.score(X_val, y_val):.3f}')

lm_reg.fit(X_train_scaled, y_train)
print(f'Ridge Regression val R^2: {lm_reg.score(X_val_scaled, y_val):.3f}')

lm_poly.fit(X_train_poly, y_train)
print(f'Degree 2 polynomial regression val R^2: {lm_poly.score(X_val_poly, y_val):.3f}')

In [ ]:
# Here is the success of our model
X_scaled = scaler.fit_transform(X.values)
lm_reg.fit(X_scaled,y)
print(f'Ridge Regression test R^2: {lm_reg.score(X_test_scaled, y_test):.3f}')